In [1]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, split

sc = pyspark.SparkContext('local[*]')

sqlContext = SQLContext(sc)


In [2]:
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('/Users/monisha/Desktop/Arizona_10users.csv')
df.columns

['user_id',
 'average_stars',
 'compliment_cool',
 'compliment_cute',
 'compliment_funny',
 'compliment_hot',
 'compliment_list',
 'compliment_more',
 'compliment_note',
 'compliment_photos',
 'compliment_plain',
 'compliment_profile',
 'compliment_writer',
 'cool',
 'fans',
 'funny',
 'review_count',
 'useful',
 'yelping_since_years',
 'friends_count',
 'elite_count']

In [16]:
df.count()

15

In [22]:
df['average_stars','compliment_cool','compliment_cute','compliment_funny','compliment_hot'].describe().show()

+-------+------------------+------------------+-----------------+------------------+------------------+
|summary|     average_stars|   compliment_cool|  compliment_cute|  compliment_funny|    compliment_hot|
+-------+------------------+------------------+-----------------+------------------+------------------+
|  count|                15|                15|               15|                15|                15|
|   mean|3.7213333333333334|105.06666666666666|6.733333333333333|105.06666666666666|63.266666666666666|
| stddev|0.9597162279003301| 401.9538826899472|26.07808786446327| 401.9538826899472| 243.0977071839536|
|    min|              2.29|                 0|                0|                 0|                 0|
|    max|               5.0|              1558|              101|              1558|               942|
+-------+------------------+------------------+-----------------+------------------+------------------+



In [23]:
df['compliment_list','compliment_more','compliment_note','compliment_photos','compliment_plain'].describe().show()

+-------+------------------+-----------------+------------------+------------------+------------------+
|summary|   compliment_list|  compliment_more|   compliment_note| compliment_photos|  compliment_plain|
+-------+------------------+-----------------+------------------+------------------+------------------+
|  count|                15|               15|                15|                15|                15|
|   mean| 7.533333333333333|              7.4|              56.4| 75.13333333333334| 53.53333333333333|
| stddev|29.176474541429208|27.04440792474481|213.75981180487867|289.60881662454034|201.83687425622654|
|    min|                 0|                0|                 0|                 0|                 0|
|    max|               113|              105|               829|              1122|               783|
+-------+------------------+-----------------+------------------+------------------+------------------+



In [25]:
df['compliment_profile','compliment_writer','cool','fans','funny'].describe().show()

+-------+------------------+------------------+---------------+------------------+------------------+
|summary|compliment_profile| compliment_writer|           cool|              fans|             funny|
+-------+------------------+------------------+---------------+------------------+------------------+
|  count|                15|                15|             15|                15|                15|
|   mean|               7.2| 42.93333333333333|          388.4|              45.6|             390.2|
| stddev| 27.33704968928223|163.26160718598962|1488.7894315277|170.01756211806895|1500.7336396385795|
|    min|                 0|                 0|              0|                 0|                 0|
|    max|               106|               633|           5770|               660|              5815|
+-------+------------------+------------------+---------------+------------------+------------------+



In [33]:
df['review_count','useful','yelping_since_years','friends_count','elite_count'].describe().show()

#new = first_five.rdd.map(lambda x: (x[0],x[1],x[2]))

+-------+------------------+-----------------+-------------------+-----------------+------------------+
|summary|      review_count|           useful|yelping_since_years|    friends_count|       elite_count|
+-------+------------------+-----------------+-------------------+-----------------+------------------+
|  count|                15|               15|                 15|               15|                15|
|   mean|123.53333333333333|402.3333333333333|  5.933333333333334|322.1333333333333|               1.0|
| stddev|330.18217193947146|1517.073765353488| 2.1201976547572383|863.0848217433843|2.4784787961282104|
|    min|                 7|                0|                  2|                1|                 0|
|    max|              1293|             5886|                 11|             3409|                 8|
+-------+------------------+-----------------+-------------------+-----------------+------------------+



In [39]:
normalized_rdd = df.rdd.map(lambda x: (x[0],((x[1] - 14.3)/9.01),((x[2] - 6.73)/26.08),((x[3] - 105.07)/401.95),
                              ((x[4] - 63.27)/243.1),((x[5] - 7.53)/29.18),((x[6] - 7.4)/27.04),((x[7] - 56.4)/213.76),
                              ((x[8] - 75.13)/289.61),((x[9] - 53.54)/201.84),((x[10] - 7.2)/27.34),((x[11] - 42.93)/163.26),
                              ((x[12] - 388.4)/1488.79),((x[13] - 45.6)/170.02),((x[14] - 390.2)/1500.73),
                              ((x[15] - 123.53)/330.18),((x[16] - 402.33)/1517.07),((x[17] - 5.93)/2.12),
                              ((x[18] - 322.13)/863.08), ((x[19] - 1.0)/2.48)))

In [41]:
first_user = normalized_rdd.first()
filtered_norm_users = normalized_rdd.filter(lambda x: x != first_user)

In [48]:
#first_user

In [43]:
import math

def cosine_sim(user):
    user = list(user)
    num = 0
    den1 = 0
    den2 = 0
    for i in range(1,len(user)):
        num += user[i] * first_user[i]
        den1 += user[i] * user[i]
        den2 += first_user[i] * first_user[i]
    den1 = math.sqrt(den1)
    den2 = math.sqrt(den2)
    cos_sim = num / float(den1 * den2)
    return cos_sim

corr = filtered_norm_users.map(lambda x : (x[0], cosine_sim(x)))

In [46]:
corr.sortBy(lambda x: -x[1]).collect()

[(u'46wwNRiBGIAJuPlk8TOP_g', 0.9980377108593387),
 (u'1Q-tjPYDBaFZJouj8Ynl2g', 0.9979256248942056),
 (u'2OtKt_7AZkfPXioU317R7g', 0.9952932215548542),
 (u'3g-2-nE22LxldS3Rx5IzNg', 0.987455788787846),
 (u'1jtr65gmv4A3uqCDaaXcxQ', 0.9848218182586048),
 (u'3iybu6gnFbiJW3kskPl8-Q', 0.9614866079710036),
 (u'0JaDPuKP812uXA6VIDMPZA', 0.8027076957545067),
 (u'2zqSXXGVGmTWZ86mAq-69w', 0.7362731521937882),
 (u'3pMczoCBOSKBcqMhVim3ow', 0.7255219910612999),
 (u'4ZfcCa4m5RWvO4EFzfYm1A', 0.5621999679286984),
 (u'1M5EuCw_7tJjJVmmnLmR9g', 0.2283144326188378),
 (u'-inp099-1gsJF3KPaS-mbg', 0.16045897468392017),
 (u'3UA89mI2pN2KphXN3QXPIQ', 0.1412629208658512),
 (u'3qz_dfwbFwTQeDRzy46G8A', 0.02091567642927863)]